In [1]:
%load_ext autoreload
%autoreload 2

In [12]:
import numpy as np
from tools2 import *
from proto2 import *
from prondict import *
from sklearn.mixture import log_multivariate_normal_density

example = np.load('lab2_example.npz')['example'].item()
data = np.load('lab2_data.npz')['data']
phoneHMMs = np.load('lab2_models.npz')['phoneHMMs'].item()

In [13]:
modellist = {}
for digit in prondict.keys():
    modellist[digit] = ['sil'] + prondict[digit] + ['sil']

In [14]:
wordHMMs = {}
wordHMMs['o'] = concatHMMs(phoneHMMs,modellist['o'],'o')

In [15]:
wordHMMs['o']['transmat'][9]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])

In [16]:
la = log_multivariate_normal_density(example['lmfcc'],wordHMMs['o']['means'],wordHMMs['o']['covars'],'diag')

/usr/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)


In [7]:
np.allclose(example['obsloglik'],la)

True

In [106]:
wordHMMs['o']['covars'].shape

(9, 13)

In [10]:
wordHMMs['o'].keys()

dict_keys(['digit', 'startprob', 'means', 'covars', 'transmat'])

In [17]:
ma = forward(la,np.log(wordHMMs['o']['startprob']),np.log(wordHMMs['o']['transmat']))
print(ma.shape)

(71, 9)


/usr/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.


In [14]:
wordHMMs['o']['transmat'].shape

(10, 10)

In [16]:
example.keys()

dict_keys(['samples', 'gender', 'speaker', 'vloglik', 'obsloglik', 'logalpha', 'loggamma', 'samplingrate', 'lmfcc', 'logbeta', 'digit', 'filename', 'loglik', 'repetition'])

In [18]:
np.allclose(example['logalpha'],ma)

True

In [19]:
np.allclose(logsumexp(ma[-1,:]),example['loglik'])

True

In [20]:
viterbi(example['obsloglik'],np.log(wordHMMs['o']['startprob']),np.log(wordHMMs['o']['transmat']))

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.


(-3773.759402930079,
 array([0, 1, 1, 1, 2, 2, 2, 2, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7,
        7, 7, 7, 7, 7], dtype=int8))

In [63]:
example['vloglik']

(-3773.759402930079,
 array([0, 1, 1, 1, 2, 2, 2, 0, 0, 0, 2, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7,
        7, 7, 7, 7, 7]))

In [21]:
fa = backward(la,np.log(wordHMMs['o']['startprob']),np.log(wordHMMs['o']['transmat']))
print(fa.shape)

(71, 9)


/usr/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.


In [73]:
np.allclose(example['logbeta'],fa)

True

In [99]:
np.exp(logsumexp(example['logalpha'][-1]))

0.0

In [102]:
np.allclose(example['loggamma'],statePosteriors(example['logalpha'],example['logbeta']))

True

In [103]:
np.exp(example['loggamma'])

array([[1.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       [2.00215652e-003, 9.97745388e-001, 2.52455253e-004,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       [5.34301025e-006, 9.98181380e-001, 1.81327719e-003,
        1.31933404e-084, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       [6.84056276e-007, 9.88415580e-001, 1.15837356e-002,
        3.56740353e-071, 5.69219766e-113, 0.00000000e+000,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       [4.44351050e-005, 1.06077282e-002, 9.89347837e-001,
        4.27774810e-058, 2.64524769e-105, 5.69219766e-113,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       [1.11369861e-002, 9.41343873e-003, 9.79449575e-001,
        5.77125208e-046, 3.31432343e-091, 2.6452477

In [22]:
np.dot(np.exp(example['loggamma']).T,example['lmfcc']).shape

(9, 13)

In [23]:
logsumexp(example['loggamma']).shape

(9,)

In [27]:
logsumexp(example['loggamma']).shape

(9,)

In [32]:
new_mean,new_covars = updateMeanAndVar(example['lmfcc'],example['loggamma'])

In [34]:
new_covars.shape

(9, 13)

In [124]:
for i in range(9):
    aaa[i]/=aa[i]

In [10]:
a = np.array([[0,0,1],[0,0,0],[0,0,2]])
b = logsumexp(a,axis=1)

In [11]:
np.exp(b)

array([4.71828183, 3.        , 9.3890561 ])